In [1]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math
# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
import chromedriver_autoinstaller
# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm
# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time
# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 크롬 옵션
options = webdriver.ChromeOptions()
# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800,1200") # window-size -> 기본 : 1920,1080
# 멜론을 키자
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(5)
driver.get("https://www.melon.com/")

# 장르음악 클릭
driver.find_element_by_css_selector('#gnb_menu > ul:nth-child(1) > li.nth3 > a > span.menu_bg.menu03').click()
time.sleep(4)
# 발라드 클릭(발라드가 첫 창이라서 아마 필요 없을 듯)
# driver.find_element_by_link_text("발라드").click()
# time.sleep(2)
# 인기순으로 변경하기
driver.find_element_by_css_selector('#frm > div > div > div > a.last').click()
time.sleep(3)
for k in range(7):
    driver.find_element_by_css_selector('#pageObjNavgation > div > a.next').click()
    time.sleep(3)

# 본격적인 가사 크롤링
song_names = []
artist_names = []
lyrics = []
for k in range(3):
    for i in range(1, 51):
        A = np.random.randint(4,10)
        time.sleep(A)
        driver.find_element_by_css_selector('#frm > div > table > tbody > tr:nth-child({}) > td:nth-child(4) > div > a'.format(i)).click()
        time.sleep(1)
        titles = driver.find_elements_by_css_selector('.song_name')[0].text
        song_names.append(titles) # 노래 제목이다
        artist = driver.find_elements_by_css_selector('.artist_name')[0].text
        artist_names.append(artist) # 가수 이름이다
        time.sleep(1)
        try:
            driver.find_element_by_css_selector('#lyricArea > button > span').click()
            lyric = driver.find_elements_by_css_selector('#d_video_summary')[0].text
            lyrics.append(lyric) # 가사이다
        except:
            lyrics.append(np.nan) # 가사 없으면 빈 칸으로 될 것임
        D = np.random.randint(5,11)
        time.sleep(D)
        driver.back()
    for j in range(2, 11):
        driver.find_element_by_css_selector('#pageObjNavgation > div > span > a:nth-child({})'.format(j)).click() # 다음 숫자 클릭
        for i in range(1, 51):
            A = np.random.randint(4,10)
            time.sleep(A)
            driver.find_element_by_css_selector('#frm > div > table > tbody > tr:nth-child({}) > td:nth-child(4) > div > a'.format(i)).click()
            time.sleep(1)
            titles = driver.find_elements_by_css_selector('.song_name')[0].text
            song_names.append(titles) # 노래 제목이다
            artist = driver.find_elements_by_css_selector('.artist_name')[0].text
            artist_names.append(artist) # 가수 이름이다
            time.sleep(1)
            try:
                driver.find_element_by_css_selector('#lyricArea > button > span').click()
                lyric = driver.find_elements_by_css_selector('#d_video_summary')[0].text
                lyrics.append(lyric) # 가사이다
            except:
                lyrics.append(np.nan) # 가사 없으면 빈 칸으로 될 것임
            D = np.random.randint(5,11)
            time.sleep(D)
            driver.back()
    driver.find_element_by_css_selector('#pageObjNavgation > div > a.next').click() # '다음'을 클릭

In [3]:
#데이터 프레임 제작
dfdf = pd.DataFrame(list(zip(song_names, artist_names, lyrics)), columns = ['제목','가수','가사'])
dfdf.head()

,제목,가수,가사
0,한 (恨),얀,나를 바라보는\n너의 마음을 알아\n숨길 수록 아픈 눈빛\n느낄 수 있고\n술취한 ...
1,거짓말이라도 해서 널 보고싶어,박장현 (VROMANCE),내가 미안해\n이렇게 아플 줄도 모르고\n너에게 모질게도 대했던 바보야\n후회에 맘...
2,X (걸어온 길에 꽃밭 따윈 없었죠),청하,어디선가 넌 나를 지켜보겠지\n지키지 못한 약속 모두 다 잊은 채\n이제야 나는 제...
3,아! 옛날이여,김유하,이젠 내 곁을 떠나간\n아쉬운 그대이기에\n마음속의 그대를\n못 잊어 그려본다\n달...
4,All With You,태연 (TAEYEON),내가 그대 곁에 있어서\n행복합니다\n내가 그대 곁에 있어서\n꿈을 꿉니다\n내가 ...


In [4]:
dfdf.to_csv("melon_ballad(인기순)(71-100).csv", encoding='utf-8-sig')